In [2]:
# Url to grap info 
'https://biep-recrute.talent-soft.com/offre-de-emploi/liste-offres.aspx?mode=list&lcid=1036&facet_Region=208&Keywords=financier'

#needed variable
base_url = 'https://biep-recrute.talent-soft.com'
keywords = 'financier'
facet_Region = "208" # check on the site to get the code location
ongoing_page = 'page=1'
url = base_url + '/offre-de-emploi/liste-offres.aspx?' + ongoing_page + '&mode=list&lcid=1036&facet_Region=' + facet_Region + '&Keywords=' + keywords
until_when = datetime.datetime.strptime('1/10/2017', "%d/%m/%Y").date()
current_page = 1


In [1]:
#Imports

import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
import datetime
from math import ceil
from requests import get
from time import sleep
from random import randint

In [3]:
# func for getting the html

def get_html(link):
    # Don't run this command multiple time in a row unless you want your ip to be ban
    print('getting data from: ' + link)
    page = get(link)
    sleep(randint(8,15))
    # Get the tree version with BeautifulSoup
    return bs(page.text, 'html.parser')

In [4]:
# Get first request

html_soup = get_html(url)

# Info about the number of page

resultat = html_soup.find_all('div', class_ = 'resultat')
nb_pages = ceil(int(resultat[0].span.text.split()[0]) / 50)

getting data from: https://biep-recrute.talent-soft.com/offre-de-emploi/liste-offres.aspx?page=1&mode=list&lcid=1036&facet_Region=208&Keywords=financier


In [5]:
# Data viariable to stock all info

dates = []
ref_link = []
titles = []
status = []
locs = []
services = []

In [6]:
# Loop throught all page (1 is already load so read data before)

while True:
    # Get all the jobs on the page
    job_containers = html_soup.find_all('li', class_ = 'offerlist-item')
    
    # Loop throught all the job to get the info

    for index, info in enumerate(job_containers): 
        # Get info for the job
        info_containers = job_containers[index].find_all('li')
        date = datetime.datetime.strptime(info_containers[1].text, "%d/%m/%Y").date()
        # Don't grab outdated info
        if date > until_when:
            # Get Data for one Job
            ref_link.append(base_url + job_containers[index].a['href'])
            dates.append(date)
            titles.append(job_containers[index].a['title'])
            status.append(info_containers[2].text)
            locs.append(info_containers[3].text)
            services.append(info_containers[4].text)
    
    current_page += 1
    if current_page > nb_pages:
        break
    ongoing_page = 'page=' + str(current_page)
    url = base_url + '/offre-de-emploi/liste-offres.aspx?' + ongoing_page + '&mode=list&lcid=1036&facet_Region=' + facet_Region + '&Keywords=' + keywords
    html_soup = get_html(url)

getting data from: https://biep-recrute.talent-soft.com/offre-de-emploi/liste-offres.aspx?page=2&mode=list&lcid=1036&facet_Region=208&Keywords=financier
getting data from: https://biep-recrute.talent-soft.com/offre-de-emploi/liste-offres.aspx?page=3&mode=list&lcid=1036&facet_Region=208&Keywords=financier
getting data from: https://biep-recrute.talent-soft.com/offre-de-emploi/liste-offres.aspx?page=4&mode=list&lcid=1036&facet_Region=208&Keywords=financier
getting data from: https://biep-recrute.talent-soft.com/offre-de-emploi/liste-offres.aspx?page=5&mode=list&lcid=1036&facet_Region=208&Keywords=financier
getting data from: https://biep-recrute.talent-soft.com/offre-de-emploi/liste-offres.aspx?page=6&mode=list&lcid=1036&facet_Region=208&Keywords=financier
getting data from: https://biep-recrute.talent-soft.com/offre-de-emploi/liste-offres.aspx?page=7&mode=list&lcid=1036&facet_Region=208&Keywords=financier
getting data from: https://biep-recrute.talent-soft.com/offre-de-emploi/liste-offr

In [7]:
int(resultat[0].span.text.split()[0])

764

In [8]:
# All job to dataFrame
wanted_job = pd.DataFrame({'title': titles,
                          'date': dates,
                          'lien': ref_link,
                          'status': status,
                          'localisation': locs,
                          'service': services})
# Sort by date
wanted_job.sort_values('date', ascending=False, inplace=True)
wanted_job = wanted_job[['title', 'date',  'status', 'localisation', 'service', 'lien']]

# Put it to excel
writer = pd.ExcelWriter('jobs.xlsx')
wanted_job.to_excel(writer, 'jobs-' + keywords)
writer.save()